## **URL links for the Kobo form**

In [ ]:
data_url = "https://kc.humanitarianresponse.info/api/v1/data/yourfile.json"


## Importing the modules required for this notebook

This notebook requires pandas, geopandas and follium for the data visualization & Analysis.

In [ ]:
import requests
import pandas as pd
import os

#### Use this cell for installing the geopandas module
If you bear any problem while running the cells, Please install the required modules as per your need with similar way as of below:

In [ ]:
import geopandas as gpd
from shapely import wkt

In [ ]:
import numpy as np

In [ ]:
import folium
import matplotlib.pyplot as plt

In [ ]:
username = 'username'
password = 'password'


## Reading the data directly from Kobo server:

It will be accesing the real time data as of time of runing this code. But be careful of the fact that, Enumerators may have collected the data in their device and haven't yet submitted the data to server

In [ ]:
response = requests.get(data_url, auth=(username, password))
j = response.json()
df = pd.json_normalize(j)

In [ ]:
df.shape

In [ ]:
df.columns

### Only selecting the  Crop and Geo-cordinates data from the dataset downloaded

In [ ]:
df = df[['enumerators',
       'A/first_criteria/second_criteria/location_info/state',
       'A/first_criteria/second_criteria/location_info/district',
       'A/first_criteria/second_criteria/B/crop',
       
       'A/first_criteria/second_criteria/C/PlotGPS',
       'A/first_criteria/second_criteria/C/shape',
       'A/first_criteria/second_criteria/C/shape_area',
       'A/first_criteria/second_criteria/C/rounded_shape_area',
       '_geolocation']]


In [ ]:
# Renaming the column headings to simple names
col_rename = ['enumerators',
    'state',
    'district',
    'crop',
   
    'PlotGPS',
    'shape',
    'shape_area',
    'rounded_shape_area', 'geolocation'
]
df.columns = col_rename

## Data submitted by individual enumerators (Number of samples for each)

In [ ]:
df.groupby(['enumerators'])['crop'].count()

### Crop sample size for whole study area

In [ ]:
df['crop'].value_counts()

## Number of samples by each enumerators with crop type samples

In [ ]:
df.groupby(['enumerators','crop'])['crop'].count()

## Each Region (AOI) wise crop samples collected so far:


In [ ]:
# Ref
# https://stackoverflow.com/questions/67093514/how-can-i-convert-a-geoshape-geotrace-geopoint-to-geojson

In [ ]:
region_dist_dict = {
    '233': 'Birgunj',
    '234': 'Birgunj',
    '320': 'Kathmandu',
    '324': 'Kathmandu',
    '325': 'Kathmandu',
    '326': 'Kathmandu',
    '327': 'Kathmandu',
    '329': 'Kathmandu',
    '439': 'Pokhara',
    '440': 'Pokhara',
    '546': 'Kapilvastu',
    '550': 'Kapilvastu',
    '556': 'Dang',
    '557': 'NepalGunj',
    '558': 'NepalGunj',
    '577': 'Rukum',
    '654': 'Rukum',
    '769': 'Achham',
    '771': 'Dhangadi',
    '772': 'Dhangadi',
}


In [ ]:
df_dict ={
    'district':list(region_dist_dict.keys()),
    'AOI':list(region_dist_dict.values())
}


In [ ]:
df_reg = pd.DataFrame.from_dict(df_dict)
df = df.merge(df_reg, how='inner', on='district')

In [ ]:
count_dist_crop = df.groupby(['AOI','crop','enumerators'])['crop'].count()


In [ ]:
count_dist_crop

In [ ]:
df.groupby(['AOI',])['crop'].count()

## Point Visualization for monitoring

In [ ]:
def wkt_point(pon):
    point = "POINT ({} {})".format(pon[1], pon[0])
    return point

In [ ]:
df['point_geom'] = df['geolocation'].map(wkt_point)
# df

df['point_geometry'] = df.point_geom.apply(wkt.loads)
df.drop('point_geom', axis=1, inplace=True) #Drop WKT column


gdf_point = gpd.GeoDataFrame(df, geometry='point_geometry')
# gdf_point

# Monitoring the feild polygons for each crop

In [ ]:
from shapely.geometry import LineString, Point, Polygon
from shapely.wkt import dumps

In [ ]:
# https://python.hotexamples.com/examples/shapely.wkt/-/dumps/python-dumps-function-examples.html#0x346e25d2de439ed401c723d3f6e3e4c911cb28698b845c5bec33796af1b082ac-106,,134,
# https://github.com/Cadasta/cadasta-platform/blob/master/cadasta/xforms/utils.py

def odk_geom_to_wkt(coords):
    """Convert geometries in ODK format to WKT."""

    if coords == '':
        return ''
#     print(coords)
    if str(coords)!='nan':
        coords = coords.replace('\n', '')
        coords = coords.split(';')
        coords = [c.strip() for c in coords]
        if (coords[-1] == ''):
            coords.pop()

        if len(coords) > 1:
            # check for a geoshape taking into account
            # the bug in odk where the second coordinate in a geoshape
            # is the same as the last (first and last should be equal)
            if len(coords) > 3:
                if coords[1] == coords[-1]:  # geom is closed
                    coords.pop()
                    coords.append(coords[0])
            points = []
            for coord in coords:
                coord = coord.split(' ')
                coord = [x for x in coord if x]
                latlng = [float(coord[1]),
                          float(coord[0])]
                points.append(tuple(latlng))
            if (coords[0] != coords[-1] or len(coords) == 2):
                return dumps(LineString(points))
            else:
                return dumps(Polygon(points))
        else:
            latlng = coords[0].split(' ')
            latlng = [x for x in latlng if x]
            return dumps(Point(float(latlng[1]), float(latlng[0])))
    else:
        return np.nan

In [ ]:


def wkt_loads(x):
    try:
        return wkt.loads(x)
    except Exception:
        return None

In [ ]:
df['geom_poly'] = df['shape'].map(odk_geom_to_wkt)


In [ ]:
df_new = df
df_new['geometry'] = df_new.geom_poly.apply(wkt_loads)
df_new.drop('geom_poly', axis=1, inplace=True) #Drop WKT column
df_new = df_new.dropna(subset=['geometry'])

# Geopandas GeoDataFrame
gdf_poly = gpd.GeoDataFrame(df_new, geometry='geometry')

## Map Visualization of the survey

In [ ]:
map_vis = folium.Map(location=[28.5212389, 81.0903013], zoom_start=10, tiles="https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}",
        attr="Google",
        name="Google Satellite")

In [ ]:
# Create a geometry list from the GeoDataFrame
geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in gdf_point.point_geometry]

# Iterate through list and add a marker for each volcano, color-coded by its type.
i = 0
for coordinates in geo_df_list:

    # Place the markers with the popup labels and data
    map_vis.add_child(
        folium.Marker(
            location=coordinates,
            popup=
                "Crop: " + str(gdf_point.crop[i]) + "<br>"
                + "Enumerator: " + str(gdf_point.enumerators[i]) + "<br>"
            ),
        )
    i = i + 1

In [ ]:
for _, r in gdf_poly.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    folium.Popup(r['enumerators']).add_to(geo_j)
    geo_j.add_to(map_vis)


## Kobo points and polygon being collected
Visualization of the sample points and polygon overlayed on google hybrid map

In [ ]:

map_vis